In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


### Load Data

In [35]:
import pandas as pd
import numpy as np

data = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")

In [36]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [37]:
# extract score and text
df = data[['Score', 'Text']]
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [38]:
# drop missing rows
df.dropna(inplace=True)
# remove score == 3 (neutral)
df = df[df.loc[:, ('Score')] != 3 ]
df['Postively rated'] = np.where(df.loc[:, ('Score')] > 3, 1, 0)
df.head()

/tmp/ipykernel_32/1529179422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Score,Text,Postively rated
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


In [39]:
df['Postively rated'].mean()

0.843980951439102

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Postively rated'], random_state=0)

print('[ X Train first Entry] \n', X_train[0])

print('\n\n [X Train shape]', X_train.shape)

[ X Train first Entry] 
 I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


 [X Train shape] (394360,)


### CountVectorizer
Convert X_train to numerical representation that Scikit-learn can use

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

In [42]:
# get vocaburary
vect.get_feature_names_out()[::2000]

array(['00', '269', '911drugstore', 'airbrushing', 'appeasement',
       'b0001dmt1e', 'b000xkcdvg', 'b002wf5jw0', 'bale', 'bionaturæ',
       'brinkman', 'captives', 'chick', 'coldbrew', 'copycats', 'dad',
       'deterioration', 'donateing', 'eloctrolytes', 'exclaims',
       'fingerfuls', 'frisking', 'glom', 'gustibus', 'hirsh',
       'impediments', 'irresistably', 'kicap', 'leslie', 'mackays',
       'meiji', 'monocle', 'nestley', 'offsets', 'paesano', 'pestering',
       'positing', 'puffiness', 'recharged', 'retreating', 'sami',
       'serving', 'slimfast', 'spoilable', 'subordinated', 'tangled',
       'timberwolf', 'tupperwares', 'upping', 'wakin', 'wont'],
      dtype=object)

In [43]:
len(vect.get_feature_names_out())

101796

In [44]:
# transform the x_train to a matrix document
X_train_vectorized = vect.transform(X_train)

In [45]:
X_train_vectorized

<394360x101796 sparse matrix of type '<class 'numpy.int64'>'
	with 21009141 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [48]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print("[ AUC ]: ", roc_auc_score(y_test, predictions))

[ AUC ]:  0.8583022160191638


In [51]:
feature_names = np.array(vect.get_feature_names_out())
sorted_coeff_index = model.coef_[0].argsort()
print('Smallest Coeff: \n', feature_names[sorted_coeff_index[:10]])
print('\n\nLargest Coeff: \n', feature_names[sorted_coeff_index[-11:-1]])

Smallest Coeff: 
 ['disappointing' 'worst' 'yuck' 'disappointment' 'tasteless' 'sounded'
 'undrinkable' 'returning' 'cancelled' 'skip']


Largest Coeff: 
 ['delicious' 'terrific' 'favorites' 'yum' 'addicted' 'refreshing'
 'worried' 'beat' 'skeptical' 'pleasantly']


### Tfidf
allows us to weigh terms based on how important ther are to document

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names_out())

35146

In [56]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print("[ AUC ]: ", roc_auc_score(y_test, predictions))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[ AUC ]:  0.8497004151642256


## n-grams

In [57]:
vect =CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names_out())

526712

In [59]:

model = LogisticRegression(max_iter=10000)
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print("[ AUC ]: ", roc_auc_score(y_test, predictions))

[ AUC ]:  0.931176118339143


In [60]:
feature_names = np.array(vect.get_feature_names_out())
sorted_coeff_index = model.coef_[0].argsort()
print('Smallest Coeff: \n', feature_names[sorted_coeff_index[:10]])
print('\n\nLargest Coeff: \n', feature_names[sorted_coeff_index[-11:-1]])

Smallest Coeff: 
 ['two stars' 'not worth' 'not recommend' 'disappointing' 'not impressed'
 'disappointment' 'not good' 'terrible' 'don recommend' 'undrinkable']


Largest Coeff: 
 ['amazing' 'four stars' 'not bad' 'not bitter' 'excellent' 'delicious'
 'just what' 'not too' 'just right' 'be disappointed']


In [61]:
print(model.predict(vect.transform(['not good food', 'she liked the food'])))

[0 1]
